In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
t = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')
t.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [3]:
t.drop(['text_id'],axis = 1,inplace = True)
t.head()

,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [4]:
ft = t.full_text
ft.head()

0    I think that students would benefit from learn...
1    When a problem is a change you have to let it ...
2    Dear, Principal\n\nIf u change the school poli...
3    The best time in life is when you become yours...
4    Small act of kindness can impact in other peop...
Name: full_text, dtype: object

In [5]:
y = t
y.drop(['full_text'],axis = 1,inplace = True)


In [6]:
ft = ft.str.lower()

In [7]:
def remove_n_r(text):
    n = []
    for t in text:
        if t not in ['\n','\r']:
                     n.append(t)
    return "".join(n)

In [8]:
ft = ft.apply(remove_n_r)

In [9]:
from nltk.corpus import stopwords

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
sw= stopwords.words('english')

In [11]:
def remove_sw(text):
    m = []
    for t in text.split():
        if t not in sw:
            m.append(t)
    return " ".join(m)

In [12]:
ft = ft.apply(remove_sw)

In [13]:
import string
punc = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',punc))
ft = ft.apply(remove_punc)

In [14]:
# from textblob import TextBlob
# def spell_correct(text):
#     tb = TextBlob(text)
#     return tb.correct().string




In [15]:
#ft = ft.apply(spell_correct)

Bag of words (1 gram, 2gram, both 1n2 grams)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform(ft).toarray()
len(cv.vocabulary_)

36136

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv2 = CountVectorizer(ngram_range = (1,2))
bow2 = cv2.fit_transform(ft).toarray()
len(cv2.vocabulary_)

388776

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv3 = CountVectorizer(ngram_range = (2,2))
bow3 = cv3.fit_transform(ft).toarray()
len(cv3.vocabulary_)

352640

In [19]:
xn1 = pd.DataFrame(bow)
xn1.columns = xn1.columns.astype(str)

In [20]:
test = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')
test.drop(['text_id'],axis = 1, inplace = True)
xt = test.full_text
xt

0    when a person has no experience on a job their...
1    Do you think students would benefit from being...
2    Thomas Jefferson once states that "it is wonde...
Name: full_text, dtype: object

In [21]:
xt = xt.str.lower()
xt = xt.apply(remove_n_r)
xt = xt.apply(remove_sw)
xt = xt.apply(remove_punc)
bow_xt = cv.transform(xt).toarray()
bow_xt.shape

(3, 36136)

In [22]:
xt1 = pd.DataFrame(bow_xt)
xt1.columns = xt1.columns.astype(str)

In [23]:
test1 = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')
test1 = pd.DataFrame(test1.text_id)
test1

,text_id
0,0000C359D63E
1,000BAD50D026
2,00367BB2546B


In [24]:
from sklearn.ensemble import RandomForestRegressor
Rf = RandomForestRegressor(max_depth = 3)
Rf.fit(xn1,y)

RandomForestRegressor(max_depth=3)

In [25]:
bowN1 = pd.DataFrame(Rf.predict(xt1),columns = y.columns)
bowN1

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,3.315754,3.224021,3.407031,3.326537,3.260235,3.265009
1,3.151384,3.054056,3.250885,3.154627,3.107460,3.098086
2,3.221700,3.123195,3.329539,3.226080,3.105180,3.166579


In [26]:
submission = pd.concat([test1,bowN1],axis = 1)
submission.reset_index(drop= True, inplace = True)
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.315754,3.224021,3.407031,3.326537,3.260235,3.265009
1,000BAD50D026,3.151384,3.054056,3.250885,3.154627,3.107460,3.098086
2,00367BB2546B,3.221700,3.123195,3.329539,3.226080,3.105180,3.166579


In [27]:
submission.to_csv('submission.csv')